<a href="https://colab.research.google.com/github/god7i11a/pynb/blob/main/SST_anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources
# https://climatereanalyzer.org/clim/sst_daily/
# https://www.pyngl.ucar.edu/
# https://www.ncl.ucar.edu/External/
# https://climatedataguide.ucar.edu/

# why $5\sigma$ ???:    # https://home.cern/resources/faqs/five-sigma

# TODOs:
## explore other ways of quantifying variations besides daily distribution around the 20 year mean.

In [ ]:
%matplotlib inline
%matplotlib ipympl


try:
    from google.colab import userdata, output
    output.enable_custom_widget_manager()
    ! pip install ipympl
    ! sudo apt-get install cm-super dvipng texlive-latex-extra texlive-latex-recommended
    COLAB = True
except ModuleNotFoundError:
    COLAB=False

from climate_reanalysis import SST, T2, get_data, construct_colormap, display_main_plot, display_jumps, display_shifting_averages, display_as_one_curve, display_latest_year_data

# get the latest data

# global average   https://climatereanalyzer.org/clim/t2_daily/json/era5_world_t2_day.json
# gridded data https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/

# compute mean and standard deviations and display raw data

In [ ]:
get_data(SST)
display_latest_year_data(SST)

# Colormaps
# https://colorcet.holoviz.org/
# https://cmasher.readthedocs.io/user/usage.html#accessing-colormaps

In [ ]:
w = construct_colormap(show_chart=True)
display(w)

## graph

In [ ]:
display_main_plot(SST, w)

# SST jumps

In [ ]:
display_jumps(SST, w)

# ONE CURVE

In [ ]:
display_as_one_curve(SST, w)

# Just checking my stuff against Climate Reanalyzer graph

In [ ]:
display_shifting_averages(SST)